In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [51]:
#read data, X = n x k
wheatX = pd.read_csv('wheatX.csv', sep=";")
wheatX.head()
#1280 spalten --> wähle Anzahl kleiner als 1280

,Unnamed: 0,wPt.0538,wPt.8463,wPt.6348,wPt.9992,wPt.2838,wPt.8266,wPt.1100,wPt.0653,wPt.4418,...,c.408290,c.408294,c.408330,c.408336,c.408375,c.408393,c.408422,c.408424,c.408426,c.408443
0,1,0,1,1,1,1,0,1,1,1,...,1,0,1,1,1,0,0,0,0,1
1,2,1,1,1,1,1,0,1,1,0,...,1,1,0,0,0,0,1,1,1,1
2,3,1,1,1,1,1,0,1,1,0,...,1,1,0,0,0,0,1,1,1,1
3,4,0,1,1,1,1,0,1,1,1,...,1,1,1,1,0,0,1,0,1,0
4,5,0,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,0,0,1,1


In [52]:
# Simulation modell y = xp *betap + epsilon
def simulate_linear_model(X, p, s):
    n, k = X.shape

    # Zufällige Auswahl von p Spalten aus Xp
    selected_columns = np.random.choice(k, p, replace=False)
    X_p = X.iloc[:, selected_columns]

    # uiv standardnormalverteilte beta_p
    beta_p = np.random.normal(0, 1, p)

    # standardabweichung von X_p
    std_X_p = np.std(X_p, axis=0)

    # epsilon mit Standardabweichung*s
    epsilon = np.random.normal(0, np.mean(std_X_p) * s, size=(n, 1))

    # Simulation Y = X_p * beta_p + epsilon
    Y = X_p.dot(beta_p) + epsilon.flatten() #matrix vektor produkt + epsilon als Vektor

    return Y, X_p, beta_p, epsilon


p = 5  # ausgewählten Spalten
s = 1  # wert für standardabweichuing epsilon

# Simulation des Modells mit wheatX als input
simulated_Y, simulated_X_p, simulated_beta_p, simulated_epsilon = simulate_linear_model(wheatX, p, s)

# Ausgabe der simulierten Werte
print("Simulierte Werte:")
print("Y:", simulated_Y)
print("X_p:")
print(simulated_X_p)
print("beta_p:", simulated_beta_p)
print("epsilon:", simulated_epsilon)

Simulierte Werte:
Y: 0     -0.988054
1     -2.585718
2     -2.971365
3     -2.568960
4     -2.757301
         ...   
594   -2.381706
595   -3.220957
596   -0.936897
597   -0.354911
598   -1.142589
Length: 599, dtype: float64
X_p:
     wPt.8492  wPt.3130  c.345836  wPt.1048  c.304842
0           0         0         1         1         0
1           1         0         1         0         0
2           1         0         1         0         0
3           1         0         1         0         1
4           1         0         0         1         1
..        ...       ...       ...       ...       ...
594         1         0         0         0         1
595         1         0         1         0         1
596         0         0         1         0         1
597         0         0         1         0         1
598         0         0         1         0         1

[599 rows x 5 columns]
beta_p: [-2.21359175  0.73634703 -0.47766545 -0.66098547 -0.17885127]
epsilon: [[ 1.50596879e-01]


In [53]:
# Zufällige Auswahl des Train-Test-Splits
min_train_size = 1  # Mindestanzahl der Trainingsdaten
max_train_size = len(wheatX)  # Maximale Anzahl der Trainingsdaten
train_size = np.random.randint(min_train_size, max_train_size + 1)

# Aufteilen der Daten
X_train, X_test, Y_train, Y_test = train_test_split(wheatX, simulated_Y, train_size=train_size, random_state=42)

# Überprüfen der Form der aufgeteilten Daten
print("Anzahl der Trainingsdaten:", len(X_train))
print("Anzahl der Testdaten:", len(X_test))

#testen
model = LinearRegression()
model.fit(X_train, Y_train)

# Vorhersagen auf dem Testdatensatz
predictions = model.predict(X_test)

# Berechnung des Mean Squared Error (MSE)
mse = mean_squared_error(Y_test, predictions)
print("Mean Squared Error (MSE) auf dem Testdatensatz:", mse)

Anzahl der Trainingsdaten: 232
Anzahl der Testdaten: 367
Mean Squared Error (MSE) auf dem Testdatensatz: 0.8345067175216132


In [54]:
# Simulation des Modells
simulated_Y, X_p, true_beta_p, _ = simulate_linear_model(wheatX, p, s)

# Festlegen der Anzahl zusätzlicher Spalten q - p
q_minus_p = 5
additional_columns = np.random.choice(list(set(range(len(wheatX.columns))) - set(X_p.columns)), q_minus_p, replace=False)
X_q = pd.concat([X_p, wheatX.iloc[:, additional_columns]], axis=1)

# Anpassen des linearen Modells
model = LinearRegression()
model.fit(X_q, simulated_Y)

# Vorhersagen auf dem Testdatensatz
predictions = model.predict(X_q)

# (a) Mittlere quadratische Abweichung (MSE)
mse = mean_squared_error(simulated_Y, predictions)
print("Mittlere quadratische Abweichung (MSE) auf dem Testdatensatz:", mse)

# (b) Korrelation zwischen geschätzten und wahren y-Werten
correlation = np.corrcoef(simulated_Y, predictions)[0, 1]
print("Korrelation zwischen geschätzten und wahren y-Werten:", correlation)

# (c) teil 1
# Theoretische Varianz der Schätzer von beta_p nach definition von linearen modell
covariance_matrix = np.linalg.inv(X_q.T @ X_q) #X^T * X da nur die Diagonaleinträge für die Berechnung betrachtet werden
theoretical_variances = np.diag(covariance_matrix)
mean_theoretical_variances = np.mean(theoretical_variances)
print("Mittlere theoretische Varianz der Schätzer von beta_p:", mean_theoretical_variances)

# teil 2: Empirische quadratische Abweichung von geschätzten beta_p zu den wahren beta_p
# Berücksichtige nur die wahren beta_p (p Elemente), nicht die zusätzlichen (q - p) Elemente
empirical_squared_errors = (model.coef_[:p] - true_beta_p)**2
mean_empirical_squared_errors = np.mean(empirical_squared_errors)
print("Mittlere empirische quadratische Abweichung von geschätzten beta_p zu den wahren beta_p:", mean_empirical_squared_errors)

#note: berücksichtige, dass das my = beta0 ist und automatisch mitberechnet wird


Mittlere quadratische Abweichung (MSE) auf dem Testdatensatz: 0.17520035351871566
Korrelation zwischen geschätzten und wahren y-Werten: 0.8852080053849236
Mittlere theoretische Varianz der Schätzer von beta_p: 0.01618492743610158
Mittlere empirische quadratische Abweichung von geschätzten beta_p zu den wahren beta_p: 0.0012741528577522399


# Aufgabe 2

oben links: geschätztemittlere abweichung von geschätzem zu wahren y --> mit test = trainings plot --> Abweichung gegen 0, indikator für overfitting (corr = 1 --> lin abhg)
mit steigender anzahl der variablen wird blup vorhersage schlechter
großes modell, was klein schätzen soll (möglichst viele variablen ungleich 0) --> penalisierungsidee
plottet nur gut, da 1:1 training und direkt auf testdaten angewandt


unterer: 50 - 50

zweiter Plot MSE für fixed rffect model recht groß,für kqs und blupbleibt er, corr sinkt


warum sinnvoll, random effects für p << n zu verwenden?

wenn p nahe n, dann erlativ hohe varianz (Xpxn)*(Xnxp)--> steigt in Bild

warum nicht nur correlation anschauen: corr sagt nur lin abhägnigkeit, nichts über absolulte werte, overfitting

